# Transformations and Actions

Es gibt nur diese zwei verschiedene Operationen auf RDDs. Es ist wichtig, dass wir diese verstehen und auseinanderhalten können.

## Transformationen

* Eine Operation welche aus einem RDD einen neuen RDD erzeugt.
* Sie bauen den Lineage Graph auf

Bespiele sind:

* Lesen einer Datei
* Mappen einer Spalte in einen anderen Wert
* Filter eines RDDs (nur Personen aus Hamburg)

Sie werden **lazy** ausgeführt: erst wenn eine Action-Operation ausgeführt wird

![](RDD-lineage-graph.dio.svg)

## Actions

* Gibt das Ergebniss von RDD-Anwendungen aus
* Stößt die Anwendung von Transformationen über den Lineage-Graph an

Bespiele sind:

* Schreibe eine Ausgabe auf die Festplatte
* Gebe die Ausgabe auf dem Bildschirm aus
* Gib die Anzahl aus


In [2]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = (
    SparkSession
        .builder
        .appName("RDD-Basic-Operations")
        .master("local[4]")
        .getOrCreate()
)

sc = spark.sparkContext

spark

23/09/10 07:56:01 WARN Utils: Your hostname, pupil-a resolves to a loopback address: 127.0.1.1; using 5.75.165.200 instead (on interface eth0)
23/09/10 07:56:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/10 07:56:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# erzeuge RDD mit Hilfe des Spark Kontext

numbers_rdd = sc.parallelize([1, 2, 3, 4, 5])

Wurde die Operation wirklich ausgeführt?

Nein: sie wurde nur dem Lineage Graph hinzugefügt.

Wir können das in der Spark-UI überprüfen.

In [5]:
# Nun geben wir den RDD aus

numbers_rdd.collect()

[1, 2, 3, 4, 5]

In der Spark-UI sehen wir nun **1** Job und **4** Tasks.

In [6]:
# Nun das erste Element ausgeben

numbers_rdd.first()

1

Wir sehen in der Spark UI einen neuen Job mit genau einem Task.

In [7]:
# Nun führen wir mehere Operationen hintereinander aus

taxi_zones_rdd = sc.textFile("TaxiZones.csv")

# aufsplitten
taxi_zones_with_cols_rdd = taxi_zones_rdd.map(lambda zone: zone.split(","))

# Pair RDD über die Bezirke

taxi_zones_by_bezirk = taxi_zones_with_cols_rdd.map(lambda zone: (zone[1],1))

# aufsummieren
taxi_zones_count_bezirke = taxi_zones_by_bezirk.reduceByKey( lambda a, b: a + b)

# filtern

only_big_bezirke = taxi_zones_count_bezirke.filter( lambda row: row[1] > 10 )


In [8]:
only_big_bezirke.collect()

[('Manhattan', 69),
 ('Brooklyn', 61),
 ('Queens', 69),
 ('Bronx', 43),
 ('Staten Island', 20)]

Nun in der Spark-UI prüfen was passiert ist...

## Narrow Transformations

Wir erinnern uns: es gibt zwei Arten von Transformationen. Narrow und Wide. Wir schauen uns nun Narrow Transformations an.

**Definition**

Eine Narrow Transformation ist eine Transformation mit der für eine Output-Partition nur genau eine Eingabe Partition genutzt wird.

![](RDD-narrow.dio.svg)

Beispiele:
* Map
* MapPartition
* Filter
* Sample
* Union
* ...

**Eigenschaften**

* Die Anzahl der Eingabe Partitionen ist in der Regel gleich der der Ausgabe-Partitionen.
* Sehr schnell. Zwischen den Partitionen werden keine Daten ausgetauscht.


## Wide Transformation


**Definition**

Eine Wide-Transformationen besteht aus zwei Schritten und führt eine sogenannte Shuffle-Partition ein.

(GroupBySum)

![](RDD-wide.dio.svg)

Was können wir hier sehen?

* Es entstehen zwei Stages(Schritte) mit jeweils zwei Tasks
* Es entsteht eine Serialisierung
* Der letzte Schritt ist wieder eine Narrow-Transformation
* Wir sehen hier auch klar das bei der Wide-Transformation eine Partition mehrfach verwendet wurde um eine Output-Partition zu erzeugen

**Bespiele**

* ReduceByKey
* Distinct
* GroupBy
* OrderBy
* Distinct
* Intersection
* ....

**Eigenschaften**

* Daten müssen zwischen Partitionen hin und herbewegt werden
* Teure Operation, Daten müssen geschrieben und gelesen werden (Disk/Network/..)
* Es entsteht ein weiterer Schritt. Dieser muss erst komplett beendet werden, dann kann mit dem nächsten begonnen werden
* Anzahl Shuffle-Partitionen kann von den Eingangspartitionen abweichen


# Jobs, Stages and Tasks

Nun möchten wir uns ein paar grundsätzliche Konzepte von Spark anschauen, um besser verstehen zu können was dort passiert.

![](RDD-Jobs-Stages-Tasks.dio.svg)

**Beachte**

* Anzahl der Jobs ist gleich der Anzahl der Action Operation
* Job 2 macht hier Zeugs doppelt! Das liegt an der Lazyness der Operationen. Dies kann man optimieren, wir sparen das für diesen Kurs aber aus.

Lass uns das nun ganz konkret an einem praktischen Beispiel sehen.

## Erster Schritt ist das Lesen der Datei

In [9]:
zones_rdd = sc.textFile("TaxiZones.csv", 4)
zones_rdd.collect()

['1,EWR,Newark Airport,EWR',
 '2,Queens,Jamaica Bay,Boro Zone',
 '3,Bronx,Allerton/Pelham Gardens,Boro Zone',
 '4,Manhattan,Alphabet City,Yellow Zone',
 '5,Staten Island,Arden Heights,Boro Zone',
 '6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone',
 '7,Queens,Astoria,Boro Zone',
 '8,Queens,Astoria Park,Boro Zone',
 '9,Queens,Auburndale,Boro Zone',
 '10,Queens,Baisley Park,Boro Zone',
 '11,Brooklyn,Bath Beach,Boro Zone',
 '12,Manhattan,Battery Park,Yellow Zone',
 '13,Manhattan,Battery Park City,Yellow Zone',
 '14,Brooklyn,Bay Ridge,Boro Zone',
 '15,Queens,Bay Terrace/Fort Totten,Boro Zone',
 '16,Queens,Bayside,Boro Zone',
 '17,Brooklyn,Bedford,Boro Zone',
 '18,Bronx,Bedford Park,Boro Zone',
 '19,Queens,Bellerose,Boro Zone',
 '20,Bronx,Belmont,Boro Zone',
 '21,Brooklyn,Bensonhurst East,Boro Zone',
 '22,Brooklyn,Bensonhurst West,Boro Zone',
 '23,Staten Island,Bloomfield/Emerson Hill,Boro Zone',
 '24,Manhattan,Bloomingdale,Yellow Zone',
 '25,Brooklyn,Boerum Hill,Boro Zone',
 '26,Brooklyn,

Überlege wieviele

 * Jobs
 * Transformationen
 * Actions
 * Stages

erzeugt wurden?

**Für die Grübler** ersetze collect() durch take(3) und schaue dir in der Spark-UI die Anzahl der Tasks an...

## Splitting mit Map

In [10]:
zones_rdd = sc.textFile("TaxiZones.csv", 4)

zones_with_cols_rdd = zones_rdd.map(lambda x: x.split(","))

zones_with_cols_rdd.collect()

[['1', 'EWR', 'Newark Airport', 'EWR'],
 ['2', 'Queens', 'Jamaica Bay', 'Boro Zone'],
 ['3', 'Bronx', 'Allerton/Pelham Gardens', 'Boro Zone'],
 ['4', 'Manhattan', 'Alphabet City', 'Yellow Zone'],
 ['5', 'Staten Island', 'Arden Heights', 'Boro Zone'],
 ['6', 'Staten Island', 'Arrochar/Fort Wadsworth', 'Boro Zone'],
 ['7', 'Queens', 'Astoria', 'Boro Zone'],
 ['8', 'Queens', 'Astoria Park', 'Boro Zone'],
 ['9', 'Queens', 'Auburndale', 'Boro Zone'],
 ['10', 'Queens', 'Baisley Park', 'Boro Zone'],
 ['11', 'Brooklyn', 'Bath Beach', 'Boro Zone'],
 ['12', 'Manhattan', 'Battery Park', 'Yellow Zone'],
 ['13', 'Manhattan', 'Battery Park City', 'Yellow Zone'],
 ['14', 'Brooklyn', 'Bay Ridge', 'Boro Zone'],
 ['15', 'Queens', 'Bay Terrace/Fort Totten', 'Boro Zone'],
 ['16', 'Queens', 'Bayside', 'Boro Zone'],
 ['17', 'Brooklyn', 'Bedford', 'Boro Zone'],
 ['18', 'Bronx', 'Bedford Park', 'Boro Zone'],
 ['19', 'Queens', 'Bellerose', 'Boro Zone'],
 ['20', 'Bronx', 'Belmont', 'Boro Zone'],
 ['21', 'Brookl

Überlege wieviele

 * Jobs
 * Transformationen
 * Actions
 * Stages

erzeugt wurden?

In [11]:
# Hat sich die Anzahl der Partitionen im Laufe der Transformationen verändert?

print("Nach der ersten Transformation: " + str(zones_rdd.getNumPartitions()))
print("Nach map: " + str(zones_with_cols_rdd.getNumPartitions()))

Nach der ersten Transformation: 4
Nach map: 4


## Gruppieren

In [12]:
# vorarbeiten zum gruppieren

zones_rdd = sc.textFile("TaxiZones.csv", 4)

zones_with_cols_rdd = zones_rdd.map(lambda x: x.split(","))

zones_by_borrow_rdd = zones_with_cols_rdd.map(lambda x: (x[1],1))

zones_by_borrow_rdd.count()

265

Überlege wieviele

 * Jobs
 * Transformationen
 * Actions
 * Stages

erzeugt wurden?

In [13]:
zones_rdd = sc.textFile("TaxiZones.csv", 4)

zones_with_cols_rdd = zones_rdd.map(lambda x: x.split(","))

zones_by_borrow_rdd = zones_with_cols_rdd.map(lambda x: (x[1],1))
grouped_rdd = zones_by_borrow_rdd.reduceByKey(lambda x,y: x+y)
grouped_rdd.collect()

[('Bronx', 43),
 ('Staten Island', 20),
 ('EWR', 1),
 ('Manhattan', 69),
 ('Brooklyn', 61),
 ('Unknown', 2),
 ('Queens', 69)]

Überlege wieviele

 * Jobs
 * Transformationen
 * Actions
 * Stages

erzeugt wurden?

(In der Spark-UI sich die Stages mal genauer anschauen (In/Out))

In [14]:
# Das ist jetzt praktisch Job2 aus unserem Beispiel
zones_rdd = sc.textFile("TaxiZones.csv", 4)

zones_with_cols_rdd = zones_rdd.map(lambda x: x.split(","))

zones_by_borrow_rdd = zones_with_cols_rdd.map(lambda x: (x[1],1))
grouped_rdd = zones_by_borrow_rdd.reduceByKey(lambda x,y: x+y)
filtered_group = grouped_rdd.filter(lambda x: x[0] == "Bronx")
filtered_group.collect()

[('Bronx', 43)]

Überlege wieviele

 * Jobs
 * Transformationen
 * Actions
 * Stages

erzeugt wurden?

(In der Spark-UI sich die Stages mal genauer anschauen (In/Out))

In [15]:
# Das ist jetzt praktisch Job1 und Job2 aus unserem Beispiel
zones_rdd = sc.textFile("TaxiZones.csv", 4)
zones_with_cols_rdd = zones_rdd.map(lambda x: x.split(","))
zones_by_borrow_rdd = zones_with_cols_rdd.map(lambda x: (x[1],1))
print(zones_by_borrow_rdd.count()) ## Job 1 aus dem Beispiel
grouped_rdd = zones_by_borrow_rdd.reduceByKey(lambda x,y: x+y)
filtered_group = grouped_rdd.filter(lambda x: x[0] == "Bronx")
filtered_group.collect() # Job 2 aus dem Beispiel

265


[('Bronx', 43)]

Überlege wieviele

 * Jobs
 * Transformationen
 * Actions
 * Stages

erzeugt wurden?

(In der Spark-UI sich die Stages mal genauer anschauen (In/Out))

## Take Aways

* Es gibt Narrow und Wide-Transformations
* Wide-Transformations sind teuer
  * fügen eine weitere Stage hinzu
  * führen in der Regel zu einer Teilserialisierung
* Jede Action für zu einem Job